In [1]:
import numpy as np
from collections import deque
import random
import GeometryDashEnvironments
from GeometryDashAgent import DQNAgent

In [ ]:
env = GeometryDashEnvironments.GeometryDashEnv()
state_shape = (84, 84, 3)
action_size = env.action_space.n
agent = DQNAgent(state_shape, action_size)
episodes=20

batch_size=32
memory = deque(maxlen=2000)
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, *state.shape])
    done = False
    total_reward = 0
    i = 0

    while not done:
        i+=1
        action = agent.act(state, epsilon)
        print(f'time: {i}, episode: {e}/{episodes}')
        next_state, reward, done, _ = env.step(action)
        print(f'reward: {reward}, done: {done}')
        next_state = np.reshape(next_state, [1, *next_state.shape])
        memory.append((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward

        if len(memory) > batch_size:
            minibatch = random.sample(memory, batch_size)
            for s, a, r, s_next, d in minibatch:
                agent.train(s, a, r, s_next, d)
                
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    print(f"Episode: {e}/{episodes}, Score: {total_reward}, Epsilon: {epsilon}")

In [4]:
agent.save("geometry_dash_dqn.weights.h5")
env.close()